# Adding new data to our dataset
Last week we spoke about "tidy data", which is a means of organizing data such that samples are rows and features are columns.

Why do we do this? We'll see in a bit how this lets us visualize complex data very quickly, as well as do things like data analysis very easily. Another benefit is that it lets us combine datasets much more easily.

Below we'll scrape some country data from the web. It will be taken from the website [nationmaster.com](nationmaster.com), which collects all kinds of data about countries. Remember that we already have our gapminder dataset from before. Let's see if we can add a little more information there.

# Create code for downloading the data
We'll use a python package called `requests` to download the data itself. This package lets us ask websites for information, and is really useful for scraping data from the web.

We'll also use a package called `BeautifulSoup`. This gives us a way of quickly parsing HTML (which is the code behind a website) and will let us pull the values from tables on [nationmaster.com](nationmaster.com).

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [3]:
# Define a function that will pull the data
def pull_page_data(url):
    # Make the request
    resp = requests.get(url)
    html = bs(resp.text)

    # Find the countries table on this page
    tables = html.find_all('table')
    countries = tables[0]
    
    # Define headers
    rows = countries.find_all('tr')
    headers = rows[0]
    headers = [ii.text.lower() for ii in headers.find_all('th')]

    # Pull the data
    data = [[ii.text.strip().strip('=') for ii in row.find_all('td')] for row in rows[1:]]
    data = pd.DataFrame(data, columns=headers)
    data['country'] = data['country'].str.lower()
    return data

In [4]:
# Run this function on two datasets
urls = ["http://www.nationmaster.com/country-info/stats/Crime/Violent-crime/Murder-rate#-date",
        "http://www.nationmaster.com/country-info/stats/Government/Government-type"]

# run the function on each dataset
murder_rate = pull_page_data(urls[0])
gov_type = pull_page_data(urls[1])

/Users/choldgraf/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/choldgraf/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
murder_rate.head()

,#,country,amount,date,graph,history
0,1,brazil,"40,974",2010,,
1,2,india,"40,752",2009,,
2,3,mexico,"25,757",2010,,
3,4,ethiopia,"20,239",2008,,
4,5,indonesia,"18,963",2008,,


In [6]:
gov_type.head()

,country,description
0,afghanistan,Islamic republic
1,albania,parliamentary democracy
2,algeria,republic
3,american samoa,NA
4,andorra,parliamentary democracy (since March 1993) tha...


# Cleaning up our data
Remember, we always want to clean up / standardize our data so that it's easy to make comparisons. Below we'll do some general steps to make our data easier to manage. Luckily for us, the data is already in "tidy" format so we don't need to change much.

# Clean up murder rate
For murder rate, we'll rename the `date` column to `update`, reflecting the fact that this date tells us the last time that murder rate was updated. We'll also rename the `amount` column so that it's more expressive of its true value.

In [7]:
rename_dict = {'date': 'murder_rate_updated',
               'amount': 'murder_rate'}
murder_rate = murder_rate[['country', 'amount', 'date']].rename(columns=rename_dict)

In [8]:
# Notice the data type of this column
murder_rate['murder_rate'][:5]

0    40,974
1    40,752
2    25,757
3    20,239
4    18,963
Name: murder_rate, dtype: object

In [9]:
# Now convert murder rate to a floating point
murder_rate['murder_rate'] = murder_rate['murder_rate'].\
    str.replace(',', '').apply(pd.to_numeric)
murder_rate['murder_rate'][:5]

0    40974.0
1    40752.0
2    25757.0
3    20239.0
4    18963.0
Name: murder_rate, dtype: float64

## Clean up govt type
Let's take a look at the type of government for each country.

In [10]:
gov_type['description'].values[:30]

array(['Islamic republic', 'parliamentary democracy', 'republic', 'NA',
       "parliamentary democracy (since March 1993) that retains as its chiefs of state a coprincipality; the two princes are the president of France and bishop of Seu d'Urgell, Spain, who are represented in Andorra by the coprinces' representatives",
       'republic; multiparty presidential regime', 'NA',
       'Antarctic Treaty Summary - the Antarctic region is governed by a system known as the Antarctic Treaty System; the system includes: 1. the Antarctic Treaty, signed on 1 December 1959 and entered into force on 23 June 1961, which establishes the legal framework for the management of Antarctica, 2. Recommendations and Measures adopted at meetings of Antarctic Treaty countries, 3. The Convention for the Conservation of Antarctic Seals (1972), 4. The Convention for the Conservation of Antarctic Marine Living Resources (1980), and 5. The Protocol on Environmental Protection to the Antarctic Treaty (1991); the 3

We can see that some of these are quite long. We could try to infer the "actual" description from what's here, but for the sake of speed, let's just cut off the long ones.

In [12]:
def cut_long_desc(description):
    return description[:20]
gov_type['description'] = gov_type['description'].apply(cut_long_desc)

## Merge them together
Now that we've got the data from these two sources into a similar format, we can easily combined them using the pandas `merge` command.

In [15]:
merged = pd.merge(murder_rate, gov_type, on='country')

In [16]:
merged.head()

,country,murder_rate,murder_rate_updated,description
0,brazil,40974.0,2010,federal republic
1,india,40752.0,2009,federal republic
2,mexico,25757.0,2010,federal republic
3,ethiopia,20239.0,2008,federal republic
4,indonesia,18963.0,2008,republic


And in these few steps we've gone from two datasets to one! Notice that there are different ways in which we can merge datasets. These are often described with the language of "set theory". Here are some ways to do this:

* Inner - only keep rows where the "join" column exists in both datasets
* Outer - keep all rows, regardless of whether a "join" column exists in both datasets
* Left - only keep rows corresponding to the "join" column in the left dataset
* Right - only keep rows corresponding to the "join" column in the right dataset

For example:

In [17]:
# Note that doing an outer product creates a bunch of `NaN`s
pd.merge(murder_rate, gov_type, on='country', how='outer')

,country,murder_rate,murder_rate_updated,description
0,brazil,40974.0,2010,federal republic
1,india,40752.0,2009,federal republic
2,mexico,25757.0,2010,federal republic
3,ethiopia,20239.0,2008,federal republic
4,indonesia,18963.0,2008,republic
5,nigeria,18422.0,2008,federal republic
6,south africa,15940.0,2010,republic
7,colombia,15459.0,2010,republic; executive
8,russia,14574.0,2010,federation
9,pakistan,13860.0,2011,federal republic


# Saving our data for later
Now that we've got some new data, we'll save this into our cleaned data folder for later use.

In [18]:
merged.to_csv(
    '../projects/gapminder/data/02_cleaned/country_data_clean.csv',
    index=False)